In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import statistics
from heapq import nlargest
from heapq import nsmallest
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
import datetime as dt
from statsmodels.tsa.arima_model import ARIMA

In [2]:
data = pd.read_csv('FORECAST_clean.csv',sep=',')
# dfs = dict(tuple(data.groupby('API_code'))) # Every API as a dictionary key

In [3]:
data['DATE'] = pd.to_datetime(data['DATE'])
data['Week_year'] = data.DATE.dt.strftime('%Y-%V')
data['Month_year'] = data.DATE.dt.strftime('%Y-%m')
# Convert the weekly aggregated data
data_weekly = data.groupby(['API_fam_code', 'Week_year']).sum()
data_weekly = data_weekly.drop(['API_code'], axis = 1 )
data_weekly_dict = dict(tuple(data_weekly.groupby('API_fam_code')))
# Convert the monthly aggregated data
data_monthly = data.groupby(['API_fam_code', 'Month_year']).sum()
data_monthly = data_monthly.drop(['API_code'], axis = 1 )
data_monthly_dict = dict(tuple(data_monthly.groupby('API_fam_code')))

In [5]:
#len_monthly = len(data_monthly_dict['JUG01A4158P109'])
data_weekly_dict.keys()

dict_keys(['JUG01A4158P109', 'JUG01B4384P109', 'JUG01B4384P1659', 'JUG01B4614P109', 'JUG01C0142P109', 'JUG01C4142P109', 'JUG01E4357C1299', 'JUG01E4357C139', 'JUG01E4357P109', 'JUG01F3362P109', 'JUG01I5090C1459', 'JUG01I6090G109', 'JUG01I6090L029', 'JUG01I6090L0329', 'JUG01I6090L049', 'JUG01I6090P109', 'JUG01P3108P1059', 'JUG01P3616P1059', 'JUG01P4108P109', 'JUG01P5613P109', 'JUG01Q1104P109', 'JUG01R1355C129', 'JUG01R3355C1359', 'JUG01R3355L109', 'JUG01R3355P109', 'JUG01S3193L0389', 'JUG01S3193P109', 'JUG01S3193P149', 'JUG01S3193P159', 'JUG01S3193P1759', 'JUG01S4193P159', 'JUG01S5193L0389', 'JUG01U-194L0019', 'JUG01U1194L0259', 'JUG01U1194L029', 'JUG01U1194L039', 'JUG01U1194L0399', 'JUG01U1194L0449', 'JUG01U1194P149', 'JUG01U1194P1659', 'JUG01U1194S149', 'JUG01U2194L049', 'JUG01U3194L0449', 'JUG01U3194P1159', 'JUG01U3194P149', 'JUG01U3194P1529', 'JUG01U3194P1659', 'JUG01U3610P109', 'JUG01U4194C209', 'JUG01U4194L0399', 'JUG01U4194P1659', 'JUG01U4194P179', 'JUG01U4600P109', 'JUG01U5172P14

In [5]:
no_zeros_week = []
for key, df in data_weekly_dict.items():
    no_week = df['SOLD'].astype(bool).sum(axis=0) / 261
    no_zeros_week.append(no_week)

API_Families_week = list(data_weekly_dict.keys())

no_zeros_month = []
for key, df in data_monthly_dict.items():
    no_month = df['SOLD'].astype(bool).sum(axis=0) / len_monthly
    no_zeros_month.append(no_month)
    
API_Families_month = list(data_monthly_dict.keys())

In [6]:
list_of_zeros_week = pd.DataFrame(
    {'API_family': API_Families_week,
     'Not_Zeros': no_zeros_week
    })

list_of_zeros_month = pd.DataFrame(
    {'API_family': API_Families_month,
     'Not_Zeros': no_zeros_month
    })

In [7]:
list_of_zeros_week.sort_values(by = 'Not_Zeros', ascending = False).head(20)

,API_family,Not_Zeros
44,JUG01U3194P149,0.896552
28,JUG01S3193P159,0.888889
61,JUG01U7703P109,0.750958
15,JUG01I6090P109,0.724138
46,JUG01U3194P1659,0.720307
8,JUG01E4357P109,0.613027
56,JUG01U5602P109,0.559387
38,JUG01U1194P149,0.555556
47,JUG01U3610P109,0.463602
36,JUG01U1194L0399,0.459770


In [8]:
list_of_zeros_month.sort_values(by = 'Not_Zeros', ascending = False).head(20)

,API_family,Not_Zeros
44,JUG01U3194P149,1.000000
61,JUG01U7703P109,0.983333
28,JUG01S3193P159,0.983333
15,JUG01I6090P109,0.966667
8,JUG01E4357P109,0.966667
38,JUG01U1194P149,0.966667
36,JUG01U1194L0399,0.950000
46,JUG01U3194P1659,0.933333
56,JUG01U5602P109,0.900000
24,JUG01R3355P109,0.883333


In [9]:
list_of_zeros_week_dict = dict(zip(list_of_zeros_week.API_family, list_of_zeros_week.Not_Zeros))
list_of_zeros_month_dict = dict(zip(list_of_zeros_month.API_family, list_of_zeros_month.Not_Zeros))

In [10]:
# Loops these from the end of the code
# data['no_week'] = data['API_fam_code'].map(list_of_zeros_week_dict)
# data['no_month'] = data['API_fam_code'].map(list_of_zeros_month_dict)

In [14]:
weekly_API_families = list_of_zeros_week[list_of_zeros_week['Not_Zeros'] > 0.80] 
weekly_API_families = weekly_API_families.drop(['Not_Zeros'], axis = 1)
weekly_API_families = list(weekly_API_families['API_family'])
weekly_API_families

['JUG01S3193P159', 'JUG01U3194P149']

In [12]:
monthly_API_families = list_of_zeros_month[list_of_zeros_month['Not_Zeros'] > 0.80] 
monthly_API_families = monthly_API_families.drop(['Not_Zeros'], axis = 1)
monthly_API_families = list(monthly_API_families['API_family'])
monthly_API_families = [x for x in monthly_API_families if x not in weekly_API_families]
monthly_API_families

['JUG01E4357P109',
 'JUG01I6090P109',
 'JUG01Q1104P109',
 'JUG01R3355P109',
 'JUG01U1194L0399',
 'JUG01U1194P149',
 'JUG01U3194P1659',
 'JUG01U3610P109',
 'JUG01U5602P109',
 'JUG01U7703P109']

In [13]:
weekly_forecast = {key: data_weekly_dict[key] for key in weekly_API_families}
monthly_forecast = {key: data_monthly_dict[key] for key in monthly_API_families}

In [14]:
#Weekly forecaster
# Uses data weekly forecast

import warnings
from sklearn.metrics import mean_squared_error

p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)

forecast_rolling_week = [] #The rolling forecasts for the API_Families for the last 26 weeks of the test period, always in the chunks of 26 per API_family
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
# prepare trainin]g dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
# make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
# calculate out of sample error
    error = sqrt(mean_squared_error(test, predictions))
    return error

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    for t in range(0,26):
        model = ARIMA(dataset[0:210 + t], best_cfg)
        model_fit = model.fit()
        forecast_t = model_fit.predict(start = 236 + t , end = 236 + t)[0]
        forecast_rolling.append(forecast_t)

        
# evaluate parameters
warnings.filterwarnings("ignore")
for dataset in weekly_forecast:
    evaluate_models(weekly_forecast[dataset]['SOLD'].values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=4778016.801
ARIMA(0, 0, 1) MSE=4311210.452
ARIMA(0, 0, 2) MSE=4208641.943
ARIMA(0, 1, 0) MSE=5773853.224
ARIMA(0, 1, 1) MSE=2967324.243
ARIMA(0, 2, 0) MSE=17766059.718
ARIMA(0, 2, 1) MSE=5822180.060
ARIMA(1, 0, 0) MSE=4131276.710
ARIMA(1, 0, 1) MSE=3114837.426
ARIMA(1, 0, 2) MSE=3120954.630
ARIMA(1, 1, 0) MSE=4311110.579
ARIMA(1, 1, 1) MSE=2971373.984
ARIMA(1, 1, 2) MSE=2986456.672
ARIMA(1, 2, 0) MSE=9957085.080
ARIMA(2, 0, 0) MSE=3877698.639
ARIMA(2, 0, 1) MSE=3113212.647
ARIMA(2, 0, 2) MSE=3144251.704
ARIMA(2, 1, 0) MSE=3764192.173
ARIMA(2, 1, 1) MSE=3065425.856
ARIMA(2, 1, 2) MSE=3062133.892
ARIMA(2, 2, 0) MSE=7532796.721
ARIMA(4, 0, 0) MSE=3278013.218
ARIMA(4, 0, 1) MSE=3230634.770
ARIMA(4, 0, 2) MSE=3143176.119
ARIMA(4, 1, 0) MSE=3261879.387
ARIMA(4, 1, 1) MSE=3188967.516
ARIMA(4, 1, 2) MSE=3232408.815
ARIMA(4, 2, 0) MSE=5090075.646
ARIMA(4, 2, 1) MSE=3311771.327
ARIMA(4, 2, 2) MSE=3056345.993
ARIMA(6, 0, 0) MSE=3233100.687
ARIMA(6, 0, 1) MSE=3079701.144
ARIMA(6

In [15]:
#Monthly forecaster
# Uses data monthly forecast

import warnings
from sklearn.metrics import mean_squared_error

p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)

forecast_rolling_month = [] #The rolling forecasts for the API_Families for the last 26 weeks of the test period, always in the chunks of 26 per API_family
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
# prepare trainin]g dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
# make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
# calculate out of sample error
    error = sqrt(mean_squared_error(test, predictions))
    return error

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    for t in range(0,6):
        model = ARIMA(dataset[0:48 + t], best_cfg)
        model_fit = model.fit()
        forecast_t = model_fit.predict(start = 54 + t , end = 54 + t)[0]
        forecast_rolling.append(forecast_t)
        

# evaluate parameters
warnings.filterwarnings("ignore")

for dataset in monthly_forecast: 
    evaluate_models(monthly_forecast[dataset]['SOLD'].values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=4139951.653
ARIMA(0, 0, 1) MSE=4086087.920
ARIMA(0, 0, 2) MSE=4178771.851
ARIMA(0, 1, 0) MSE=11040177.751
ARIMA(0, 1, 1) MSE=4305182.940
ARIMA(0, 1, 2) MSE=4848619.741
ARIMA(0, 2, 0) MSE=37908497.444
ARIMA(0, 2, 1) MSE=11586958.910
ARIMA(1, 0, 0) MSE=4091466.456
ARIMA(1, 0, 1) MSE=3792337.452
ARIMA(1, 0, 2) MSE=4232916.106
ARIMA(1, 1, 0) MSE=6022329.536
ARIMA(1, 1, 1) MSE=4342054.279
ARIMA(1, 1, 2) MSE=7819389.662
ARIMA(1, 2, 0) MSE=14638326.194
ARIMA(2, 0, 0) MSE=4271355.138
ARIMA(2, 1, 0) MSE=5333878.399
ARIMA(2, 1, 1) MSE=4922915.465
ARIMA(2, 2, 0) MSE=11179660.593
ARIMA(4, 0, 0) MSE=4455796.529
ARIMA(4, 1, 0) MSE=4967598.474
ARIMA(4, 1, 1) MSE=4935256.156
ARIMA(4, 1, 2) MSE=5811446.221
ARIMA(4, 2, 0) MSE=8638928.418
ARIMA(6, 0, 0) MSE=4944141.328
ARIMA(6, 1, 0) MSE=5551258.347
ARIMA(6, 1, 1) MSE=4965155.935
ARIMA(6, 2, 0) MSE=9339816.293
ARIMA(8, 0, 0) MSE=5585435.974
ARIMA(8, 1, 0) MSE=8192570.349
ARIMA(8, 2, 0) MSE=11967185.686
ARIMA(10, 0, 0) MSE=8524502.663
B

ARIMA(1, 1, 1) MSE=23534340.825
ARIMA(1, 2, 0) MSE=98076842.430
ARIMA(2, 0, 0) MSE=36420369.762
ARIMA(2, 1, 0) MSE=33068863.188
ARIMA(2, 2, 0) MSE=70576639.145
ARIMA(4, 0, 0) MSE=35344906.277
ARIMA(4, 1, 0) MSE=21997943.365
ARIMA(4, 1, 1) MSE=25158648.246
ARIMA(4, 2, 1) MSE=23457939.335
ARIMA(4, 2, 2) MSE=25691031.724
ARIMA(6, 0, 0) MSE=30882466.316
ARIMA(6, 1, 0) MSE=25130668.070
ARIMA(8, 0, 0) MSE=32888773.123
ARIMA(8, 0, 1) MSE=32434946.710
ARIMA(8, 1, 0) MSE=27747627.302
ARIMA(10, 0, 0) MSE=32152886.091
Best ARIMA(0, 1, 1) MSE=21434807.464
ARIMA(0, 0, 0) MSE=38393034.088
ARIMA(0, 0, 1) MSE=42175507.954
ARIMA(0, 0, 2) MSE=42597867.220
ARIMA(0, 1, 0) MSE=100670831.162
ARIMA(0, 1, 1) MSE=45036826.413
ARIMA(0, 2, 0) MSE=310239589.799
ARIMA(0, 2, 1) MSE=105848130.219
ARIMA(1, 0, 0) MSE=42234868.809
ARIMA(1, 1, 0) MSE=77721474.849
ARIMA(1, 1, 1) MSE=46104904.293
ARIMA(1, 1, 2) MSE=46358665.822
ARIMA(1, 2, 0) MSE=204307166.691
ARIMA(2, 0, 0) MSE=42859868.728
ARIMA(2, 0, 1) MSE=41054013.12

In [21]:
forecast_rolling_month = pd.DataFrame(forecast_rolling_month)
forecast_rolling_month.to_csv(r'/Users/henriksaarinen/Desktop/forecast_rolling_month_me.csv', index = False, header=True)

In [ ]:
forecast_rolling_week = pd.DataFrame(forecast_rolling_week)
forecast_rolling_week.to_csv(r'/Users/henriksaarinen/Desktop/forecast_rolling_month_me.csv', index = False, header=True)